In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection
c:/Users/PC/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp2/exp_vae"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp2/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp2/test_EDA.csv')
train

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_new,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age
0,462711,33484,62644,55.23,Salaried,1,1,0,0,0,...,62644.000000,Low,54340.127169,Low,High,54340.127169,More than One,54340.127169,NaN,12377
1,558674,66882,81187,84.37,Salaried,1,1,0,0,0,...,81187.000000,Extreme,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,11190
2,528251,59113,71757,84.87,Self employed,1,1,0,0,0,...,71757.000000,High,54340.127169,Extreme,High,54340.127169,More than One,54340.127169,NaN,11082
3,633950,56059,71307,81.34,Salaried,1,1,0,0,0,...,71307.000000,High,54340.127169,High,No History,54340.127169,One,54340.127169,NaN,19848
4,476747,56759,67100,85.69,Self employed,1,1,0,0,0,...,67100.000000,Medium,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,18051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186518,643379,55549,82230,70.78,Self employed,1,0,0,1,0,...,82230.000000,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,12178
186519,587710,60213,71857,84.89,Salaried,1,1,0,0,0,...,71857.000000,High,54340.127169,Extreme,No History,54340.127169,One,54340.127169,NaN,16195
186520,636081,52328,68757,78.54,Self employed,1,1,0,0,1,...,68757.000000,Medium,54340.127169,High,Low,54340.127169,More than One,54340.127169,One (or more),11267
186521,431351,77579,100360,79.71,Salaried,1,1,0,0,0,...,54340.127169,Extreme,54340.127169,High,Medium,54340.127169,More than One,54340.127169,One (or more),21527


In [4]:
train_con = train[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [5]:
train_bin = train [['UNIQUEID', 'EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 
                    'VOTERID_FLAG', 'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 
                    'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE',
                    'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE',
                    'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 
                    'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES', 'LOAN_DEFAULT',
                    'DISBURSED_AMOUNT_bins', 'ASSET_COST_bins', 'LTV_bins',
                    'PERFORM_CNS_SCORE_bins', 'PRI_NO_OF_ACCTS_bins', 'PRI_OVERDUE_ACCTS_bins', 'age']]

In [6]:
test_con = test[['EMPLOYMENT_TYPE', 'MOBILENO_AVL_FLAG', 'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG',
                   'DRIVING_FLAG', 'PASSPORT_FLAG', 'PERFORM_CNS_SCORE', 'PERFORM_CNS_SCORE_DESCRIPTION', 
                   'PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 
                   'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT', 
                   'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
                   'LOAN_DEFAULT', 'DISBURSED_AMOUNT_new', 'ASSET_COST_new', 
                   'LTV_new', 'age']]

In [7]:
# Confusion Matrix

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix"',
                          cmap = plt.cm.Blues) :
    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])) :
        plt.text(j, i, cm[i, j],
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [8]:
scaler_data = StandardScaler() 
def scaleColumns(df, cols_to_scale):

    for col in cols_to_scale:

        df[col] = pd.DataFrame(scaler_data.fit_transform(pd.DataFrame(train_con[col])),columns=[col])

    return df

In [9]:
y_train = train_con[['LOAN_DEFAULT']]
x_train= train_con.loc[:, train_con.columns != 'LOAN_DEFAULT']
x_train.shape

(186523, 24)

In [10]:
x_train = x_train.fillna(0)
x_train.shape

(186523, 24)

In [11]:
y_train

,LOAN_DEFAULT
0,0
1,0
2,1
3,1
4,0
...,...
186518,0
186519,0
186520,0
186521,0


In [12]:
y_train.values

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [13]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train.LOAN_DEFAULT), y=y_train.LOAN_DEFAULT)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.6378163041991519, 1: 2.3140088827134457}

In [14]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(train.drop(columns=['LOAN_DEFAULT']), train['LOAN_DEFAULT'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [15]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [16]:
from sdv.single_table import TVAESynthesizer


gan_synthesizer = TVAESynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            verbose= True,) 

gan_synthesizer.fit(oversampling_sample)
gan_synthesizer.save(
    filepath=f'{save_dir}/vae_synthesizer.pkl'
)

c:\ProgramData\miniconda3\envs\tensorflow_gpu\lib\site-packages\sdv\single_table\base.py:144: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

c:\ProgramData\miniconda3\envs\tensorflow_gpu\lib\site-packages\sdv\single_table\base.py:122: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



MemoryError: Unable to allocate 203. GiB for an array with shape (292440, 186523) and data type int32

In [ ]:
gan_synthetic_data = gan_synthesizer.sample(num_rows=len(train))
gan_synthetic_data.head()

,EVENT_ID,DISBURSED_AMOUNT,ASSET_COST,LTV,EMPLOYMENT_TYPE,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,...,ASSET_COST_bins,LTV_new,LTV_bins,PERFORM_CNS_SCORE_bins,PRI_NO_OF_ACCTS_new,PRI_NO_OF_ACCTS_bins,PRI_OVERDUE_ACCTS_new,PRI_OVERDUE_ACCTS_bins,age,LOAN_DEFAULT
0,591347,60732,72920,83.25,Self employed,1,1,0,0,0,...,High,54340.127169,High,High,54340.127169,One,54340.127169,NaN,19175,0
1,612627,57585,65642,81.74,Salaried,1,1,0,0,0,...,Medium,54340.127169,Extreme,Medium,54340.127169,One,54340.127169,NaN,14328,1
2,543631,64523,82685,75.39,Self employed,1,1,0,0,0,...,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,18374,1
3,552944,74047,109104,76.59,Self employed,1,1,0,0,0,...,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,18783,1
4,541361,56882,86389,69.55,Self employed,1,0,0,0,0,...,Extreme,54340.127169,Medium,No History,54340.127169,One,54340.127169,NaN,13257,1


In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)


from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)




Generating report ...

(1/2) Evaluating Data Validity: |██████████| 48/48 [00:00<00:00, 69.20it/s]| 
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 499.98it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 48/48 [00:03<00:00, 14.61it/s]|
Column Shapes Score: 87.0%

(2/2) Evaluating Column Pair Trends: |█▌        | 182/1128 [00:06<00:28, 32.66it/s]|

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdmetrics\column_pairs\statistical\contingency_similarity.py:47: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



(2/2) Evaluating Column Pair Trends: |███▏      | 353/1128 [00:12<00:22, 33.99it/s]|

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdmetrics\column_pairs\statistical\contingency_similarity.py:47: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



(2/2) Evaluating Column Pair Trends: |███▊      | 429/1128 [00:14<00:21, 32.53it/s]|

...

(2/2) Evaluating Column Pair Trends: |███████▏  | 808/1128 [00:25<00:09, 35.55it/s]|

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdmetrics\column_pairs\statistical\contingency_similarity.py:47: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



(2/2) Evaluating Column Pair Trends: |████████▌ | 962/1128 [00:30<00:04, 36.47it/s]|

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdmetrics\column_pairs\statistical\contingency_similarity.py:47: RuntimeWarning:

The values in the array are unorderable. Pass `sort=False` to suppress this warning.



(2/2) Evaluating Column Pair Trends: |██████████| 1128/1128 [00:35<00:00, 31.35it/s]|
Column Pair Trends Score: 89.47%

Overall Score (Average): 88.24%



In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## gan data
gan_data=gan_synthetic_data[gan_synthetic_data.LOAN_DEFAULT==1]
gan_sample=pd.concat([train, gan_data] , axis=0, ignore_index=True)
X_gansampled, y_gansampled = rus.fit_resample(gan_sample.drop(columns=['LOAN_DEFAULT']), gan_sample['LOAN_DEFAULT'])
X_gansampled = pd.DataFrame(X_gansampled)
y_gansampled = pd.DataFrame(y_gansampled)
gan_sample=pd.concat([X_gansampled, y_gansampled], axis=1)    
gan_sample = shuffle(gan_sample, random_state=42)
print(gan_sample.LOAN_DEFAULT.value_counts())

LOAN_DEFAULT
0    139723
1    139723
Name: count, dtype: int64


In [ ]:
label_encoders = {}
for column in gan_sample.select_dtypes(include=['object']).columns:
    label_encoder = LabelEncoder()
    gan_sample[column] = label_encoder.fit_transform(gan_sample[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

gan_sample.head()

In [ ]:
gan_sample = scaleColumns(gan_sample,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

gan_sample.head()

In [ ]:
gan_sample.to_csv(f'{save_dir}/data_train.csv', index=False)
x_train = gan_sample.drop(columns=['LOAN_DEFAULT'])
y_train = gan_sample['LOAN_DEFAULT']

In [ ]:
from sklearn.model_selection import train_test_split
x_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

+ X_train: 279446
Fold 0: 
+ train_idx: [     0      1      2 ... 279443 279444 279445]
+ valid_idx: [     6     11     12 ... 279419 279420 279430]
+ train / valid: [     6     11     12 ... 279419 279420 279430]
Fold 1: 
+ train_idx: [     0      2      3 ... 279440 279441 279444]
+ valid_idx: [     1      4     31 ... 279442 279443 279445]
+ train / valid: [     1      4     31 ... 279442 279443 279445]
Fold 2: 
+ train_idx: [     1      2      4 ... 279443 279444 279445]
+ valid_idx: [     0      3     20 ... 279426 279429 279438]
+ train / valid: [     0      3     20 ... 279426 279429 279438]
Fold 3: 
+ train_idx: [     0      1      3 ... 279442 279443 279445]
+ valid_idx: [     2      7      8 ... 279435 279440 279444]
+ train / valid: [     2      7      8 ... 279435 279440 279444]
Fold 4: 
+ train_idx: [     0      1      2 ... 279443 279444 279445]
+ valid_idx: [     5      9     10 ... 279436 279439 279441]
+ train / valid: [     5      9     10 ... 279436 279439 279441]


In [ ]:
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
scaled_df = scaleColumns(test_con,['PERFORM_CNS_SCORE','PRI_ACTIVE_ACCTS', 'PRI_CURRENT_BALANCE', 
                                 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_OVERDUE_ACCTS', 
                                 'SEC_CURRENT_BALANCE', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
                                 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'CREDIT_HISTORY_LENGTH', 
                                 'NO_OF_INQUIRIES', 'DISBURSED_AMOUNT_new',
                                 'ASSET_COST_new', 'LTV_new', 'age'])

test_dummy = pd.get_dummies(scaled_df, prefix_sep='_', drop_first=True)
y_test = test_dummy[['LOAN_DEFAULT']]
x_test= test_dummy.loc[:, test_dummy.columns != 'LOAN_DEFAULT']
x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)